# PART 1: PHPMyAdmin interface @ https://sql.springboard.com/ (SQL syntax)

/* QUESTIONS

/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */
SELECT DISTINCT name
  FROM  Facilities
 WHERE membercost > 0;


/* Q2: How many facilities do not charge a fee to members? */
SELECT COUNT(DISTINCT facid) from Facilities where membercost = 0;

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */
SELECT DISTINCT facid, name, membercost, monthlymaintenance
	FROM Facilities
	WHERE membercost > 0 AND membercost < monthlymaintenance * 0.2;


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */
SELECT * FROM Facilities WHERE facid IN (1, 5);


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */
SELECT name, monthlymaintenance,
	CASE
		WHEN monthlymaintenance >= 100 THEN "cheap"
		ELSE "expensive"
	END as label
FROM Facilities;

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */
SELECT firstname, surname
	FROM Members
	WHERE joindate IN (SELECT MAX(joindate)
                       FROM Members);


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */
SELECT DISTINCT name, CONCAT(firstname, ' ', surname) AS MemberName
	FROM Bookings AS b
		INNER JOIN Facilities AS f ON b.facid = f.facid
		INNER JOIN Members AS m ON b.memid = m.memid
	ORDER BY MemberName;


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT name as Facility,
	CASE
		WHEN m.surname = 'GUEST' THEN m.surname
		ELSE CONCAT(firstname, ' ', surname)
		END AS Name,
	CASE
		WHEN m.memid = 0 THEN b.slots * f.guestcost
		ELSE b.slots * f.membercost
		END AS Cost
  FROM Facilities as f
  	INNER JOIN Bookings as b ON b.facid = f.facid
  	INNER JOIN Members as m ON b.memid = m.memid
  WHERE b.starttime LIKE "2012-09-14%"
  	AND
  		CASE
  			WHEN m.memid = 0 THEN b.slots * f.guestcost
  			ELSE b.slots * f.membercost
  			END > 30
  ORDER BY Cost DESC;


/* Q9: This time, produce the same result as in Q8, but using a subquery. */
SELECT *
FROM (
  SELECT f.name AS Facility,
          CASE
            WHEN m.surname = 'GUEST' THEN m.surname
            ELSE CONCAT(firstname, ' ', surname)
            END AS Name,
          CASE
            WHEN m.memid = 0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
            END AS Cost
  FROM Facilities as f
  	INNER JOIN Bookings as b ON b.facid = f.facid
  	INNER JOIN Members as m ON b.memid = m.memid
  WHERE b.starttime LIKE "2012-09-14%"
)sub
WHERE sub.Cost >30
ORDER BY sub.Cost DESC;

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

SELECT f.name, 
	sum(CASE 
		WHEN b.memid = 0 THEN b.slots * f.guestcost
		ELSE b.slots * f.membercost
		END) AS revenue
FROM Facilities AS f
	INNER JOIN Bookings AS b ON b.facid = f.facid
GROUP BY f.name
HAVING revenue < 1000
ORDER BY revenue;

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

SELECT CONCAT(m.firstname, ' ', m.surname) as member,
	   CONCAT(r.firstname, ' ', r.surname) as recommended_by
FROM Members as m
	INNER JOIN Members as r ON m.recommendedby = r.memid
WHERE m.recommendedby != 0
ORDER BY r.surname, r.firstname;


/* Q12: Find the facilities with their usage by member, but not guests */
SELECT f.name as facility,
	 COUNT(mb.memid) AS usage_member
FROM (SELECT facid, memid
     	FROM Bookings 
     	WHERE memid != 0) as mb
    INNER JOIN Facilities as f ON mb.facid = f.facid
GROUP BY mb.facid;


/* Q13: Find the facilities usage by month, but not guests */
SELECT f.name as facility,
		mb.month as month,
		COUNT(mb.memid) as member_usage
FROM (
	SELECT MONTH(starttime) as month, memid, facid
	FROM Bookings
	WHERE memid != 0) as mb
	INNER JOIN Facilities as f ON f.facid = mb.facid
GROUP by mb.month;


In [1]:
# PART 2: SQLite syntax
query_1 = """
        SELECT *
        FROM FACILITIES
        """

'''
Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 
'''

query_10 = """
        SELECT f.name, 
            sum(CASE 
                WHEN b.memid = 0 THEN b.slots * f.guestcost
                ELSE b.slots * f.membercost
                END) AS revenue
        FROM Facilities AS f
            INNER JOIN Bookings AS b ON b.facid = f.facid
        GROUP BY f.name
        HAVING revenue < 1000
        ORDER BY revenue;
        """

'''Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order '''
query_11 = """
        SELECT (m.firstname || ' ' || m.surname) as member,
               (r.firstname || ' ' || r.surname) as recommended_by
        FROM Members as m
            INNER JOIN Members as r ON m.recommendedby = r.memid
        WHERE m.recommendedby != 0
        ORDER BY r.surname, r.firstname;
        """


'''Q12: Find the facilities with their usage by member, but not guests'''
query_12 = """
        SELECT f.name as facility,
             COUNT(mb.memid) AS usage_member
        FROM (SELECT facid, memid
                FROM Bookings 
                WHERE memid != 0) as mb
            INNER JOIN Facilities as f ON mb.facid = f.facid
        GROUP BY mb.facid;
        """


'''Q13: Find the facilities usage by month, but not guests'''
query_13 = """
    SELECT f.name as facility,
            mb.month as month,
            COUNT(mb.memid) as member_usage
    FROM (
        SELECT strftime('%m', starttime) as month, memid, facid
        FROM Bookings
        WHERE memid != 0) as mb
        INNER JOIN Facilities as f ON f.facid = mb.facid
    GROUP by mb.month;
    """

In [2]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)

    return conn


def select_all_tasks(conn, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    cur.execute(query)

    rows = cur.fetchall()

    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"

    # create a database connection
    conn = create_connection(database)
    with conn:
        print("2. Query all tasks")
        select_all_tasks(conn, query_1)
        
        print("\n10. Query a list of facilities with a total revenue less than 1000")
        select_all_tasks(conn, query_10)
        
        print("\n11. Produce a report of members and who recommended them in alphabetic surname,firstname order")
        select_all_tasks(conn, query_11)
        
        print("\n12. Find the facilities with their usage by member, but not guests")
        select_all_tasks(conn, query_12)
        
        print("\n13. Find the facilities usage by month, but not guests")
        select_all_tasks(conn, query_13)
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)

10. Query a list of facilities with a total revenue less than 1000
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)

11. Produce a report of members and who recommended them in alphabetic surname,firstname order
('Ramnaresh Sarwin', 'Florence Bader')
('Joan Coplin', 'Timothy Baker')
('Matthew Genting', 'Gerald Butters')
('Timothy Baker', 'Jemima Farrell')
('David Pinker', 'Jemima Farrell')
('Henrietta Rumney', 'Matthew Genting')
('Douglas Jones', 'David Jones')
('Nancy Dare', 'Janice Joplette')
('David Jones', 'Janice Joplette')
('John Hunt', 'Millicent Purview')
('Tim Boothe', 'Tim Rownam')
('Jan